In [4]:
# Hack Sample Code
# This shows examples of logging in and using the subreddit, submission, redditor, ancd comments objects
# It is not implied that you need to collect these specific attributes of submissions or redditors, but are merely examples

# Imports
import praw
import pandas as pd
import numpy as np
import datetime


In [11]:

# Login and create Reddit instance
myClientIDvar = '7H-wkx3wmj96hJazvWQb-A'
myClientSecret = 'kKTtoMJA0gYXEr9d0ohMp-niRM0yhw'
myRedditUserName = 'Legitimate_Thing5210'
reddit = praw.Reddit(client_id=myClientIDvar, client_secret=myClientSecret, user_agent=myRedditUserName)


In [38]:
"""
Get the top 5 posts on Christianity
"""
subredditName = 'christianity'
subreddit_top_5 = reddit.subreddit(subredditName).hot(limit=5)

In [39]:
""""
Get Sample Dataframe of top 5 Posts' information
"""
post_lists = []

for indexPosts, submission in enumerate(subreddit_top_5):
    post_lists.append({
      'id': submission.id,
      'Sub': subredditName,
      'Title': submission.title,
      'URL': submission.permalink,
      'Time': datetime.datetime.fromtimestamp(submission.created_utc),
      'Author': str(submission.author),
      'Body': submission.selftext,
      'IsSelfPost': bool(submission.is_self)
    })

    print(submission.is_self)
    print('Retrieved data for '+str(indexPosts+1)+' posts')


dfPosts = pd.DataFrame(post_lists)
print(dfPosts)

True
Retrieved data for 1 posts
True
Retrieved data for 2 posts
False
Retrieved data for 3 posts
True
Retrieved data for 4 posts
True
Retrieved data for 5 posts
       id           Sub                                              Title  \
0  y9sd0x  christianity  Off-Topic Friday - Post nontopical things in t...   
1  yaa0xf  christianity  Please pray for a woman I know and her baby an...   
2  yag2y4  christianity  I hope this is allowed. I am severely bipolar ...   
3  ya3b79  christianity  What are things that Christians would admit mo...   
4  ya49tt  christianity                    Is the Pro Life Movement Toxic?   

                                                 URL                Time  \
0  /r/Christianity/comments/y9sd0x/offtopic_frida... 2022-10-21 05:23:13   
1  /r/Christianity/comments/yaa0xf/please_pray_fo... 2022-10-21 18:03:37   
2  /r/Christianity/comments/yag2y4/i_hope_this_is... 2022-10-21 23:25:17   
3  /r/Christianity/comments/ya3b79/what_are_thing... 2022-10-21 13

In [41]:
"""
Get Sample Dataframe of author information
"""

author_list = []
for indexAuthor, thisIndex in enumerate(dfPosts.index):
    # Get Entry Information
    thisSubmission = reddit.submission(id=dfPosts.loc[thisIndex,'id'])
    thisAuthor = thisSubmission.author

    # Get Author Information
    TempDictAuthor = {}

    # Verifies that author exists and has not been suspended (as suspended accounts do not have most attributes)
    if ((thisSubmission.author is not None) and hasattr(thisAuthor,'created_utc')): 
        TempDictAuthor['userName'] = thisAuthor.name
        TempDictAuthor['accountAge'] = thisAuthor.created_utc
        TempDictAuthor['commentKarma'] = thisAuthor.comment_karma
        TempDictAuthor['linkKarma'] = thisAuthor.link_karma
        TempDictAuthor['isMod'] = thisAuthor.is_mod
        numPosts = 0
        numCommentsList = []

        # Get number of posts by the author in the past month and the average number of comments to it
        for thisAuthorPost in thisAuthor.submissions.top(time_filter="month"):
            numPosts = numPosts + 1
            numComments = 0
            for tempComment in thisAuthorPost.comments:
                if hasattr(tempComment,'is_submitter'): # If tempComment does not have a is_submitter attribute, it is because it is a "More Comments" link
                    numComments = numComments + 1
                else:
                    numComments = np.nan # For simplicity, just made this NaN as the summation is unknown unless clicking "More Comments", which is not part of this example code
            numCommentsList.append(numComments)
        TempDictAuthor['numPosts'] = numPosts
        TempDictAuthor['avgCommentsToPost'] = np.mean(numCommentsList)
    author_list.append(TempDictAuthor) # Append to dataframe

    print('Retrieved data for '+str(indexAuthor+1)+' authors')

dfAuthors = pd.DataFrame(author_list)
dfAuthors['isMod'] = dfAuthors['isMod'].astype('bool')
print(dfAuthors)

Retrieved data for 1 authors
Retrieved data for 2 authors
Retrieved data for 3 authors
Retrieved data for 4 authors
Retrieved data for 5 authors
           userName    accountAge  commentKarma  linkKarma  isMod  numPosts  \
0   AgentSmithRadio  1.430921e+09         58429      25313   True         5   
1  _mercurial_high_  1.636953e+09          1338       1902  False        14   
2   GirlWhoLovesGod  1.654921e+09            12         93  False         9   
3      SteadfastEnd  1.527618e+09        178219      19394  False        59   
4      Salted_Mango  1.665414e+09           208        553  False        25   

   avgCommentsToPost  
0               5.80  
1               5.50  
2               2.00  
3                NaN  
4              11.08  


In [ ]:
dfAuthors
